In [15]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS,SUMO_TOTAL_SECONDS
from od.env.station import BS_PRESET
from od.social.manager import  DynamicSocialGroupBehaviour,QoSLevel
import od.misc.statistic as oms

In [7]:
print(os.getcwd())
statistics = []
for i in range(5):
        interest_config = InterestConfig(
                DynamicSocialGroupBehaviour.MAX_N_MEMBER,
                i*2+5,
                True,
                ResourceAllocatorType.NOMA_APR,
                True,
                1.4,
                1
        )
        with open("../data/POST_REGNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                statistics.append(pickle.load(file))
        print(interest_config)


/home/OD/repos/ProjectSumo/ipynb
MAX_N_MEMBER(5)-yQoS-yRSU-NOMA_APR-D1.4-S1
MAX_N_MEMBER(7)-yQoS-yRSU-NOMA_APR-D1.4-S1
MAX_N_MEMBER(9)-yQoS-yRSU-NOMA_APR-D1.4-S1
MAX_N_MEMBER(11)-yQoS-yRSU-NOMA_APR-D1.4-S1
MAX_N_MEMBER(13)-yQoS-yRSU-NOMA_APR-D1.4-S1


In [8]:
[str(stat.interest_config) for stat in statistics]

['MAX_N_MEMBER(5)-yQoS-yRSU-NOMA_APR-D1.4-S1',
 'MAX_N_MEMBER(7)-yQoS-yRSU-NOMA_APR-D1.4-S1',
 'MAX_N_MEMBER(9)-yQoS-yRSU-NOMA_APR-D1.4-S1',
 'MAX_N_MEMBER(11)-yQoS-yRSU-NOMA_APR-D1.4-S1',
 'MAX_N_MEMBER(13)-yQoS-yRSU-NOMA_APR-D1.4-S1']

In [9]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV

In [10]:
GV.NET_STATION_CONTROLLER = GV.NET_STATION_CONTROLLER[:2]
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2]

In [11]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [22]:
app_stats_fn = lambda x: { 
    header:record 
    for sg in x.social_group
    for header,record in x.sg_header[sg].items()
    if sg.qos == QoSLevel.GENERAL
}
# app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

In [23]:
bs_index = 1

In [24]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x : x.time_gen*1000000+x.time_serv)
    for helper in sorted(bs_serv_list[i],key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits):
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [ ]:
for v in bs_serv_list[0]:
    print(v)

In [ ]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


In [ ]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])
